# AIT Development notebook

## notebook of structure

| #  | Name                                               | cells | for_dev | edit               | description                                                                |
|----|----------------------------------------------------|-------|---------|--------------------|----------------------------------------------------------------------------|
| 1  | [Environment detection](##1-Environment-detection) | 1     | No      | uneditable         | detect whether the notebook are invoked for packaging or in production     |
| 2  | [Preparing AIT SDK](##2-Preparing-AIT-SDK)         | 1     | Yes     | uneditable         | download and install AIT SDK                                               |
| 3  | [Dependency Management](##3-Dependency-Management) | 3     | Yes     | required(cell #2)  | generate requirements.txt for Docker container                             |
| 4  | [Importing Libraries](##4-Importing-Libraries)     | 2     | Yes     | required(cell #1)  | import required libraries                                                  |
| 5  | [Manifest Generation](##5-Manifest-Generation)     | 1     | Yes     | required           | generate AIT Manifest                                                      |
| 6  | [Prepare for the Input](##6-Prepare-for-the-Input) | 1     | Yes     | required           | generate AIT Input JSON (inventory mapper)                                 |
| 7  | [Initialization](##7-Initialization)               | 1     | No      | uneditable         | initialization for AIT execution                                           |
| 8  | [Function definitions](##8-Function-definitions)   | N     | No      | required           | define functions invoked from Main area.<br> also define output functions. |
| 9  | [Main Algorithms](##9-Main-Algorithms)             | 1     | No      | required           | area for main algorithms of an AIT                                         |
| 10 | [Entry point](##10-Entry-point)                    | 1     | No      | uneditable         | an entry point where Qunomon invoke this AIT from here                     |
| 11 | [License](##11-License)                            | 1     | Yes     | required           | generate license information                                               |
| 12 | [Deployment](##12-Deployment)                      | 1     | Yes     | uneditable         | convert this notebook to the python file for packaging purpose             |

## notebook template revision history

1.0.1 2020/10/21

* add revision history
* separate `create requirements and pip install` editable and noeditable
* separate `import` editable and noeditable

1.0.0 2020/10/12

* new cerarion

## body

### #1 Environment detection

[uneditable]

In [1]:
# Determine whether to start AIT or jupyter by startup argument
import sys
is_ait_launch = (len(sys.argv) == 2)

### #2 Preparing AIT SDK

[uneditable]

In [2]:
if not is_ait_launch:
    # get ait-sdk file name
    from pathlib import Path
    from glob import glob
    import re
    import os

    current_dir = %pwd

    ait_sdk_path = "./ait_sdk-*-py3-none-any.whl"
    ait_sdk_list = glob(ait_sdk_path)
    ait_sdk_name = os.path.basename(ait_sdk_list[-1])

    # install ait-sdk
    !pip install -q --upgrade pip
    !pip install -q --no-deps --force-reinstall ./$ait_sdk_name

### #3 Dependency Management

#### #3-1 [uneditable]

In [3]:
if not is_ait_launch:
    from ait_sdk.common.files.ait_requirements_generator import AITRequirementsGenerator
    requirements_generator = AITRequirementsGenerator()

#### #3-2 [required]

In [4]:
if not is_ait_launch:
    requirements_generator.add_package('numpy', '1.26.3')
    requirements_generator.add_package('matplotlib', '3.7.3')
    requirements_generator.add_package('pandas', '2.2.2')
    requirements_generator.add_package('scikit-learn', '1.4.2')
    requirements_generator.add_package('tensorflow', '2.11.1')
    requirements_generator.add_package('scipy', '1.13.0')

#### #3-3 [uneditable]

In [5]:
if not is_ait_launch:
    requirements_generator.add_package(f'./{ait_sdk_name}')
    requirements_path = requirements_generator.create_requirements(current_dir)

    !pip install -q -r $requirements_path 

### #4 Importing Libraries

#### #4-1 [required]

In [6]:
# import if you need modules cell
# from pathlib import Path

import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input, Flatten, Dense, Lambda, Reshape, BatchNormalization, MaxPooling2D, Dropout
from tensorflow.keras import backend as K
import tensorflow.keras as keras
import matplotlib.pyplot as plt
from scipy.stats import norm

from collections import defaultdict 
import random
import numpy as np
from sklearn.utils import shuffle
from scipy import stats
from sklearn.metrics import precision_score, recall_score, accuracy_score 
from collections import Counter

import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef


2024-04-20 01:10:04.112402: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-20 01:10:04.225846: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-04-20 01:10:04.225865: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-04-20 01:10:04.945612: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-

#### #4-2 [uneditable]

In [7]:
# must use modules
from os import path
import shutil  # do not remove
from ait_sdk.common.files.ait_input import AITInput  # do not remove
from ait_sdk.common.files.ait_output import AITOutput  # do not remove
from ait_sdk.common.files.ait_manifest import AITManifest  # do not remove
from ait_sdk.develop.ait_path_helper import AITPathHelper  # do not remove
from ait_sdk.utils.logging import get_logger, log, get_log_path  # do not remove
from ait_sdk.develop.annotation import measures, resources, downloads, ait_main  # do not remove
# must use modules

### #5 Manifest Generation

[required]

In [8]:
if not is_ait_launch:
    from ait_sdk.common.files.ait_manifest_generator import AITManifestGenerator
    manifest_generator = AITManifestGenerator(current_dir)
    manifest_generator.set_ait_name('eval_noise_score_aquavs')
    manifest_generator.set_ait_description('モデルの安定性を評価するために、ノイズを付けたラベルで検証します。\
SVAEの潜在表現を使用し、入力データセット内の各サンプルの異常を測定する「ノイズスコア」を計測します。\
詳細については、元の論文「Pulastya, et al. Assessing the quality of the datasets by identifying mislabeled samples」\
(URL: https://dl.acm.org/doi/abs/10.1145/3487351.3488361)')

    manifest_generator.set_ait_source_repository('https://github.com/aistairc/Qunomon_AIT_eval_noise_score_aquavs')
    manifest_generator.add_ait_licenses('Apache License Version 2.0')
    manifest_generator.set_ait_version('1.0')
    manifest_generator.add_ait_keywords('Evaluation')
    manifest_generator.set_ait_quality('https://ait-hub.pj.aist.go.jp/ait-hub/api/0.0.1/qualityDimensions/機械学習品質マネジメントガイドライン第三版/C-2機械学習モデルの安定性')
    
    # inventories
    inventory_requirement_image_dataset = manifest_generator.format_ait_inventory_requirement(format_=['npz'])
    manifest_generator.add_ait_inventories(name='image_dataset', 
                                           type_='dataset', 
                                           description='画像データセット ※以下の4つのみ利用可能：mnist data, fashion mnist data, cifar10 data, cifar100 data', 
                                           requirement=inventory_requirement_image_dataset)

    # input parameters, Hyperparameters
    manifest_generator.add_ait_parameters(name='MAD_Outlier_constant', 
                                          type_='float', 
                                          default_val='1.5', 
                                          description='潜在空間における外れ値検出を指定するハイパーパラメーター')
    manifest_generator.add_ait_parameters(name='MISLABEL_THRESHOLD', 
                                          type_='float', 
                                          default_val='0.5', 
                                          description='外れ値次元の割合に基づく誤ラベルを指定するハイパーパラメーター')
    manifest_generator.add_ait_parameters(name='latent_dim', 
                                          type_='int', 
                                          default_val='100', 
                                          description='潜在空間の次元を指定するハイパーパラメーター')
    manifest_generator.add_ait_parameters(name='batch_size', 
                                          type_='int', 
                                          default_val='32', 
                                          description='VAEのオプティマイザのバッチサイズを指定するハイパーパラメーター')
    
    # input parameters
    manifest_generator.add_ait_parameters(name='datasetName', 
                                          type_='str', 
                                          default_val='mnist', 
                                          description='データセットを指定するパラメーター ※指定値について以下の4つのみ利用可能：mnist, fashion_mnist, cifar10, cifar100')
    manifest_generator.add_ait_parameters(name='noise_perc', 
                                          type_='float', 
                                          default_val='20', 
                                          description='ノイズ付きラベルの割合を指定するパラメーター')
    manifest_generator.add_ait_parameters(name='noise_systematic', 
                                          type_='str', 
                                          default_val='Sys', 
                                          description='ラベル値に基づいてノイズを加えるタイプを指定するパラメーター ※指定値についてSysまたはUni')
    manifest_generator.add_ait_parameters(name='model_name', 
                                          type_='str', 
                                          default_val='', 
                                          description='VAEモデル名称を指定するパラメーター')
    
    # measures: evaluation metrics
    manifest_generator.add_ait_measures(name='evaluation_result_accuracy', 
                                        type_='float', 
                                        structure='single', 
                                        min='0', 
                                        max='1', 
                                        description='正確度（精度）')
    manifest_generator.add_ait_measures(name='evaluation_result_precision', 
                                        type_='float', 
                                        structure='single', 
                                        min='0', 
                                        max='1', 
                                        description='適合率')
    manifest_generator.add_ait_measures(name='evaluation_result_recall', 
                                        type_='float', 
                                        structure='single', 
                                        min='0', 
                                        max='1', 
                                        description='再現率')
    manifest_generator.add_ait_measures(name='evaluation_result_f1', 
                                        type_='float', 
                                        structure='single', 
                                        min='0', 
                                        max='1', 
                                        description='F1スコア')
    manifest_generator.add_ait_measures(name='evaluation_result_roc_auc', 
                                        type_='float', 
                                        structure='single', 
                                        min='0', 
                                        max='1', 
                                        description='ROC曲線下面積（AUC）')
    manifest_generator.add_ait_measures(name='evaluation_result_mcc', 
                                        type_='float', 
                                        structure='single', 
                                        min='0', 
                                        max='1', 
                                        description='マシューズ相関係数')

    # download: VAE model
    manifest_generator.add_ait_downloads(name='vae', 
                                         description='VAE model learned')
    manifest_generator.add_ait_downloads(name='log', 
                                         description='AIT execution logs')
    manifest_path = manifest_generator.write()


### #6 Prepare for the Input

[required]

In [9]:
if not is_ait_launch:
    from ait_sdk.common.files.ait_input_generator import AITInputGenerator
    input_generator = AITInputGenerator(manifest_path)
    input_generator.add_ait_inventories(name='image_dataset',
                                        value='mnist_data/mnist_train_data.npz')

    input_generator.set_ait_params(name='MAD_Outlier_constant', value=1.5)
    input_generator.set_ait_params(name='MISLABEL_THRESHOLD', value= 0.5)
    input_generator.set_ait_params(name='latent_dim', value=100)
    input_generator.set_ait_params(name='batch_size', value=32)
    input_generator.set_ait_params(name='datasetName', value='mnist')
    input_generator.set_ait_params(name='noise_perc', value=10)
    input_generator.set_ait_params(name='noise_systematic', value='Sys')
    input_generator.set_ait_params(name='model_name', value='vae_mnist_Sys_10.keras')

    input_generator.write()

### #7 Initialization

[uneditable]

In [10]:
logger = get_logger()

ait_manifest = AITManifest()
ait_input = AITInput(ait_manifest)
ait_output = AITOutput(ait_manifest)

if is_ait_launch:
    # launch from AIT
    current_dir = path.dirname(path.abspath(__file__))
    path_helper = AITPathHelper(argv=sys.argv, ait_input=ait_input, ait_manifest=ait_manifest, entry_point_dir=current_dir)
else:
    # launch from jupyter notebook
    # ait.input.json make in input_dir
    input_dir = '/usr/local/qai/mnt/ip/job_args/1/1'
    current_dir = %pwd
    path_helper = AITPathHelper(argv=['', input_dir], ait_input=ait_input, ait_manifest=ait_manifest, entry_point_dir=current_dir)

ait_input.read_json(path_helper.get_input_file_path())
ait_manifest.read_json(path_helper.get_manifest_file_path())

### do not edit cell

### #8 Function definitions

[required]

In [11]:
### helper function
def specify_dimensions(datasetName):
    if datasetName == 'mnist' or datasetName == 'fashion_mnist':
        # constants to specify model details
        img_dimensions = (28, 28, 1)
        num_classes = 10
        num_channels = 1
    elif datasetName == 'cifar10':
        # constants to specify model details
        img_dimensions = (32, 32, 3)
        num_classes = 10
        num_channels = 3
    elif datasetName == 'cifar100':
        # constants to specify model details
        img_dimensions = (32, 32, 3)
        num_classes = 100
        num_channels = 3
    else:
        raise ValueError
    return img_dimensions, num_classes, num_channels

def prepare_data(datasetName, train_data, train_labels, noisePerc, noiseType):
    img_dimensions, num_classes, num_channels = specify_dimensions(datasetName)
    
    #adds noise to y-labels using uniform noise model - i.e. mislabeled samples are given labels uniformly at random.
    def add_noise_UniformNoiseModel(input_y, perc, allClasses):
        final_idx = defaultdict(list)
        noisy_y = [-1 for i in range(input_y.shape[0])]

        for i in range(input_y.shape[0]): 
            final_idx[input_y[i]].append(i)

        for lbl in final_idx.keys():
            remC = (perc/100.0)*len(final_idx[lbl])
            #print("Label: ", lbl, "; # of datapoints flipped: ", int(remC))
            for i in range(int(remC)):
                idx = random.randint(0, len(final_idx[lbl]) - 1)
                newLabel = random.choice(allClasses)
                while (newLabel == lbl):
                    newLabel = random.choice(allClasses)
                noisy_y[final_idx[lbl][idx]] = newLabel  # update the label for datapoint from `label` to `newLabel` 
                del final_idx[lbl][idx]

        for lbl in final_idx.keys():
            for i in final_idx[lbl]:
                noisy_y[i] = lbl

        return np.array(noisy_y)

    #adds noise to y-labels using systematic noise model - i.e. mislabeled samples are given labels systematic at random.
    def add_noise_SystematicNoiseModel(input_y, perc, allClasses):
        final_idx = defaultdict(list)
        noisy_y = [-1 for i in range(input_y.shape[0])]

        for i in range(input_y.shape[0]): 
            final_idx[input_y[i]].append(i)

        for lbl in final_idx.keys():
            remC = (perc/100.0)*len(final_idx[lbl])
            #print("Label: ", lbl, "; # of datapoints flipped: ", int(remC))
            for i in range(int(remC)):
                idx = random.randint(0, len(final_idx[lbl]) - 1)
                newLabel = (lbl + 1)%(len(allClasses))
                noisy_y[final_idx[lbl][idx]] = newLabel  # update the label for datapoint from `label` to `newLabel` 
                del final_idx[lbl][idx]

        for lbl in final_idx.keys():
            for i in final_idx[lbl]:
                noisy_y[i] = lbl

        return np.array(noisy_y)

    # min-max normalization
    def min_max_normalize(lis):
        minL = float(min(lis))
        maxL = float(max(lis))
        minMaxLis = [float((float(x) - minL)/ (maxL - minL)) for x in lis]
        return minMaxLis  
    
    #reshaping
    train_data = train_data.reshape((train_data.shape[0], img_dimensions[0], img_dimensions[1], img_dimensions[2]))
    train_labels = train_labels.reshape(train_labels.shape[0])

    # convert from integers to floats
    train_data = train_data.astype('float32')

    # normalize to range 0-1
    train_data = train_data / 255.0

    if(noiseType == "Sys"):
        noisy_labels = add_noise_SystematicNoiseModel(train_labels, noisePerc, [cl for cl in range(10)])
    elif(noiseType == "Uni"):
        noisy_labels = add_noise_UniformNoiseModel(train_labels, noisePerc, [cl for cl in range(10)])

    grn_truth = np.array(noisy_labels == train_labels, dtype=int)

    print("Number of mislabeled: ", len(grn_truth) - sum(grn_truth), "out of", len(grn_truth))

    y_enc_noisy_labels = tf.keras.utils.to_categorical(noisy_labels) #encode noisy labels

    return train_data, noisy_labels, grn_truth, y_enc_noisy_labels

# grouping datapoints by respective classes
def group_data_by_class(input_x, input_y):
    final_out = defaultdict(list) 
    final_idx = defaultdict(list)
    for i in range(input_x.shape[0]): 
        final_out[input_y[i]].append(input_x[i])
        final_idx[input_y[i]].append(i)
    return final_out, final_idx

# Ref - https://core.ac.uk/download/pdf/206095228.pdf
def outlier_detection_med_mad(input_data, k1):
    column_med = np.median(input_data, axis = 0)
    column_mad = stats.median_abs_deviation(input_data,axis = 0)

    #computing threshold for each feature
    threshold_lower = column_med - (k1*column_mad)
    threshold_upper = column_med + (k1*column_mad)
    outliers = []
    num_outlier_feature_list = []
    outlier_level = defaultdict(list)
    for i in range(input_data.shape[0]):
        num_outlier_feature = 0
        x = input_data[i]
        for id in range(x.shape[0]):
            if not (threshold_lower[id] <= x[id] and x[id] <= threshold_upper[id]):
                num_outlier_feature += 1
        outlier_level[num_outlier_feature].append(i)
    return outlier_level

# computes noise level of each datapoint 
def get_train_lvl(encoder, input_x, input_y, MAD_Outlier_constant):
    grouped_train, grouped_idx = group_data_by_class(input_x, input_y.reshape(input_y.shape[0]))
    cntr = 0
    train_lvl = [-1 for i in range(input_x.shape[0])]
    for digit in range(0,10):
        z_values = encoder.predict(np.array(grouped_train[digit]))[2]
        class_outliers = outlier_detection_med_mad(z_values, MAD_Outlier_constant)
        for i in class_outliers.keys():
            for j in class_outliers[i]:
                # i is the outlier level
                # grouped_idx[digit][j] is the index
                train_lvl[grouped_idx[digit][j]] = i
    return np.array(train_lvl)

### main functions
@log(logger)
@downloads(ait_output, path_helper, 'vae', 'vae_model_learned.keras')
def optimize_AQUAVS(datasetName, train_data, y_enc_noisy_labels, latent_dim, batch_size, model_name, file_path = None):
    img_dimensions, num_classes, num_channels = specify_dimensions(datasetName)
    
    # VAE 
    def vae_loss(data, reconstruction):
        z_mean, z_log_var, z = encoder(data)
        reconstruction_loss = keras.losses.binary_crossentropy(data, reconstruction)
        reconstruction_loss = tf.reduce_mean(reconstruction_loss, axis=[1,2])
        kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
        kl_loss = tf.reduce_mean(kl_loss, axis=1)
        kl_loss *= -0.5
        total_loss = tf.reduce_mean(reconstruction_loss + kl_loss)/100
        return total_loss

    def sampling(args):
        z_mean, z_var = args
        epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0,)  ## latent_dim = K.shape(z_mean)[1] 
        return z_mean + K.exp(z_var / 2) * epsilon

    ## ENCODER
    inputNode = Input(shape=img_dimensions, name="EncoderInput")
    enc_inter = Conv2D(filters=32, kernel_size=4, strides=2, padding='same', kernel_initializer='he_uniform')(inputNode)
    enc_inter = Conv2D(filters=64, kernel_size=4, strides=2, padding='same', kernel_initializer='he_uniform', activation='relu')(enc_inter)
    enc_inter = Conv2D(filters=128, kernel_size=4, strides=1, padding='same', kernel_initializer='he_uniform', activation=tf.nn.relu)(enc_inter)

    conv_shape = K.int_shape(enc_inter) 

    enc_inter = Flatten()(enc_inter)
    z_mean = Dense(latent_dim, name="Mean")(enc_inter)
    z_var = Dense(latent_dim, name="Variance")(enc_inter)
    z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_var])
    encoder = Model(inputNode, [z_mean, z_var, z], name="Encoder")

    ## CLASSIFIER
    clf_latent_inputs = Input(shape=(latent_dim,), name='ClassifierInput')
    clf_outputs = Dense(num_classes, activation='softmax', name='ClassifierOutput')(clf_latent_inputs)
    clf_supervised = Model(clf_latent_inputs, clf_outputs, name='Classifier')

    ## DECODER
    inputNode2 = Input(shape=(latent_dim,), name="DecoderInput")
    dec_inter = Dense(conv_shape[1]*conv_shape[2]*conv_shape[3])(inputNode2)
    dec_inter = Reshape((conv_shape[1], conv_shape[2], conv_shape[3]))(dec_inter)
    dec_inter = Conv2DTranspose(filters=128, kernel_size=4, strides=1, padding='same', kernel_initializer='he_uniform', activation='relu')(dec_inter)
    dec_inter = Conv2DTranspose(filters=64, kernel_size=4, strides=2, padding='same', kernel_initializer='he_uniform', activation='relu')(dec_inter)
    dec_inter = Conv2DTranspose(filters=32, kernel_size=4, strides=2, padding='same', kernel_initializer='he_uniform', activation='relu')(dec_inter)
    decoder_node = Conv2DTranspose(num_channels, kernel_size=4, strides=1, padding='same')(dec_inter)
    decoder = Model(inputNode2, decoder_node, name='Decoder')

    output_combined = [decoder(encoder(inputNode)[2]), clf_supervised(encoder(inputNode)[2])]
    vae = Model(inputNode, output_combined, name='S-VAE')

    vae.compile(optimizer='adam', loss=[vae_loss, 'categorical_crossentropy'])

    # callback definitions
    def scheduler(epoch):
        return 0.001/(epoch+1)

    earlyStopCallback = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        min_delta=0,
        patience=0,
        verbose=0,
        mode="auto",
        baseline=None,
        restore_best_weights=True,
    )

    lrScheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

    splitID = int(0.8*len(train_data))

    #Note - VAE trains on noisy data
    vae.fit(train_data[:splitID], [train_data[:splitID], y_enc_noisy_labels[:splitID]], 
            shuffle=True, epochs=10, batch_size=batch_size, 
            validation_data=(train_data[splitID:], [train_data[splitID:], y_enc_noisy_labels[splitID:]]), 
            callbacks=[lrScheduler, earlyStopCallback],
            verbose=1)

    model_name = file_path
    if model_name is not None:
        vae.save(model_name)
    return vae


@log(logger)
@measures(ait_output, 'evaluation_result_accuracy')
def evaluate_accuracy_AQUAVS(encoder, train_data, noisy_labels, grn_truth, MAD_Outlier_constant, MISLABEL_THRESHOLD, latent_dim):
    
    noisy_lvl = get_train_lvl(encoder, train_data, noisy_labels, MAD_Outlier_constant)

    # identify mislabel
    true_mislabels = grn_truth
    estimated_mislabels = np.where(noisy_lvl / latent_dim <= MISLABEL_THRESHOLD, 1, 0)

    # evaluate the differnce between true_mislabels vs estimated_mislabels
    return accuracy_score(true_mislabels, estimated_mislabels)

@log(logger)
@measures(ait_output, 'evaluation_result_precision')
def evaluate_precision_AQUAVS(encoder, train_data, noisy_labels, grn_truth, MAD_Outlier_constant, MISLABEL_THRESHOLD, latent_dim):
    
    noisy_lvl = get_train_lvl(encoder, train_data, noisy_labels, MAD_Outlier_constant)

    # identify mislabel
    true_mislabels = grn_truth
    estimated_mislabels = np.where(noisy_lvl / latent_dim <= MISLABEL_THRESHOLD, 1, 0)

    # evaluate the differnce between true_mislabels vs estimated_mislabels
    return precision_score(true_mislabels, estimated_mislabels)

@log(logger)
@measures(ait_output, 'evaluation_result_recall')
def evaluate_recall_AQUAVS(encoder, train_data, noisy_labels, grn_truth, MAD_Outlier_constant, MISLABEL_THRESHOLD, latent_dim):
    
    noisy_lvl = get_train_lvl(encoder, train_data, noisy_labels, MAD_Outlier_constant)

    # identify mislabel
    true_mislabels = grn_truth
    estimated_mislabels = np.where(noisy_lvl / latent_dim <= MISLABEL_THRESHOLD, 1, 0)

    # evaluate the differnce between true_mislabels vs estimated_mislabels
    return recall_score(true_mislabels, estimated_mislabels)

@log(logger)
@measures(ait_output, 'evaluation_result_f1')
def evaluate_f1_AQUAVS(encoder, train_data, noisy_labels, grn_truth, MAD_Outlier_constant, MISLABEL_THRESHOLD, latent_dim):
    
    noisy_lvl = get_train_lvl(encoder, train_data, noisy_labels, MAD_Outlier_constant)

    # identify mislabel
    true_mislabels = grn_truth
    estimated_mislabels = np.where(noisy_lvl / latent_dim <= MISLABEL_THRESHOLD, 1, 0)

    # evaluate the differnce between true_mislabels vs estimated_mislabels
    return f1_score(true_mislabels, estimated_mislabels)

@log(logger)
@measures(ait_output, 'evaluation_result_roc_auc')
def evaluate_roc_auc_AQUAVS(encoder, train_data, noisy_labels, grn_truth, MAD_Outlier_constant, MISLABEL_THRESHOLD, latent_dim):
    
    noisy_lvl = get_train_lvl(encoder, train_data, noisy_labels, MAD_Outlier_constant)

    # identify mislabel
    true_mislabels = grn_truth
    estimated_mislabels = np.where(noisy_lvl / latent_dim <= MISLABEL_THRESHOLD, 1, 0)

    # evaluate the differnce between true_mislabels vs estimated_mislabels
    return roc_auc_score(true_mislabels, estimated_mislabels)

@log(logger)
@measures(ait_output, 'evaluation_result_mcc')
def evaluate_mcc_AQUAVS(encoder, train_data, noisy_labels, grn_truth, MAD_Outlier_constant, MISLABEL_THRESHOLD, latent_dim):
    
    noisy_lvl = get_train_lvl(encoder, train_data, noisy_labels, MAD_Outlier_constant)

    # identify mislabel
    true_mislabels = grn_truth
    estimated_mislabels = np.where(noisy_lvl / latent_dim <= MISLABEL_THRESHOLD, 1, 0)

    # evaluate the differnce between true_mislabels vs estimated_mislabels
    return matthews_corrcoef(true_mislabels, estimated_mislabels)



In [12]:
@log(logger)
@downloads(ait_output, path_helper, 'Log', 'ait.log')
def move_log(file_path: str=None) -> str:
    shutil.move(get_log_path(), file_path)

### #9 Main Algorithms

[required]

In [13]:
@log(logger)
@ait_main(ait_output, path_helper)
def main() -> None:
    # inventory
    input_data = np.load(ait_input.get_inventory_path('image_dataset'))
    
    # parameters
    MAD_Outlier_constant = ait_input.get_method_param_value('MAD_Outlier_constant')
    MISLABEL_THRESHOLD = ait_input.get_method_param_value('MISLABEL_THRESHOLD')
    latent_dim = ait_input.get_method_param_value('latent_dim')
    batch_size = ait_input.get_method_param_value('batch_size')
    datasetName = ait_input.get_method_param_value('datasetName')
    noise_perc = ait_input.get_method_param_value('noise_perc')
    noise_systematic = ait_input.get_method_param_value('noise_systematic')
    model_name = ait_input.get_method_param_value('model_name')
    
    train_data, noisy_labels, grn_truth, y_enc_noisy_labels = prepare_data(datasetName, input_data['X'], input_data['y'], noise_perc, noise_systematic)
    
    vae = optimize_AQUAVS(datasetName, train_data, y_enc_noisy_labels, latent_dim, batch_size, model_name)
    encoder = vae.get_layer('Encoder')
    
    evaluation_result = evaluate_accuracy_AQUAVS(encoder, train_data, noisy_labels, grn_truth, MAD_Outlier_constant, MISLABEL_THRESHOLD, latent_dim)
    evaluation_result = evaluate_precision_AQUAVS(encoder, train_data, noisy_labels, grn_truth, MAD_Outlier_constant, MISLABEL_THRESHOLD, latent_dim)
    evaluation_result = evaluate_recall_AQUAVS(encoder, train_data, noisy_labels, grn_truth, MAD_Outlier_constant, MISLABEL_THRESHOLD, latent_dim)
    evaluation_result = evaluate_f1_AQUAVS(encoder, train_data, noisy_labels, grn_truth, MAD_Outlier_constant, MISLABEL_THRESHOLD, latent_dim)
    evaluation_result = evaluate_roc_auc_AQUAVS(encoder, train_data, noisy_labels, grn_truth, MAD_Outlier_constant, MISLABEL_THRESHOLD, latent_dim)
    evaluation_result = evaluate_mcc_AQUAVS(encoder, train_data, noisy_labels, grn_truth, MAD_Outlier_constant, MISLABEL_THRESHOLD, latent_dim)
    
    move_log()

### #10 Entry point

[uneditable]

In [ ]:
if __name__ == '__main__':
    main()

Number of mislabeled:  5996 out of 60000


2024-04-20 01:10:07.604181: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-04-20 01:10:07.604234: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-04-20 01:10:07.604255: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (2fc43c737668): /proc/driver/nvidia/version does not exist
2024-04-20 01:10:07.604506: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
1500/1500 [==============================] - 223s 147ms/step - loss: 0.6027 - Decoder_loss: 0.0149 - Classifier_loss: 0.5878 - val_loss: 0.4746 - val_Decoder_loss: 0.0145 - val_Classifier_loss: 0.4601 - lr: 0.0010
Epoch 2/10
129/184 [====================>.........] - ETA: 0s

### #11 License

[required]

In [ ]:
ait_owner='AIST'
ait_creation_year='2024'

### #12 Deployment

[uneditable] 

In [ ]:
if not is_ait_launch:
    from ait_sdk.deploy import prepare_deploy
    from ait_sdk.license.license_generator import LicenseGenerator
    
    current_dir = %pwd
    prepare_deploy(ait_sdk_name, current_dir, requirements_path)
    
    # output License.txt
    license_generator = LicenseGenerator()
    license_generator.write('../top_dir/LICENSE.txt', ait_creation_year, ait_owner)
